In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from PIL import Image

In [5]:
def check_image(path):
    try:
        img = Image.open(path)
        return True
    except:
        return False

In [4]:
img_transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [ ]:
train_data_path = 
val_data_path = 
test_data_path = 

train_data = torchvision.datasets.ImageFolder(root=train_data_path, transform=img_transforms, is_valid_file=check_image)
val_data = torchvision.datasets.ImageFolder(root=val_data_path, transform=img_transforms, is_valid_file=check_image)
test_data = torchvision.datasets.ImageFolder(root=test_data_path, transform=img_transforms, is_valid_file=check_image)

In [ ]:
BATCH_SIZE = 64

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE)
val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=BATCH_SIZE)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE)

In [9]:
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(3 * 64 * 64, 84)
        self.fc2 = nn.Linear(84, 50)
        self.fc3 = nn.Linear(50, 2)
        
    def forward(self, x):
        x = x.view(-1, 3 * 64 * 64)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [12]:
simpleNet = SimpleNet()

In [13]:
from torchsummary import summary
summary(simpleNet, (3, 64, 64))

[2022-03-19 08:41:10.647 1-8-1-cpu-py36-ml-t3-medium-62c6b413a5e8d67a1da6b0c48d04:94 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-03-19 08:41:10.875 1-8-1-cpu-py36-ml-t3-medium-62c6b413a5e8d67a1da6b0c48d04:94 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 84]       1,032,276
            Linear-2                   [-1, 50]           4,250
            Linear-3                    [-1, 2]             102
Total params: 1,036,628
Trainable params: 1,036,628
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.05
Forward/backward pass size (MB): 0.00
Params size (MB): 3.95
Estimated Total Size (MB): 4.00
----------------------------------------------------------------


In [ ]:
optimizer = optim.Adam(simpleNet.parameters(), lr=1e-3)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

simpleNet.to(device)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # Compute prediction error.
        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        
        # Backpropagation.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")